In [ ]:
!pip install facenet-pytorch

In [ ]:
!pip install mahotas

In [ ]:
import os
import mahotas
import glob
import time
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import copy
import json
from google.colab import drive

# See github.com/timesler/facenet-pytorch:
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

In [ ]:
from facenet_pytorch.models.inception_resnet_v1 import get_torch_home
torch_home = get_torch_home()

In [ ]:
drive.mount('/content/drive')

In [ ]:
images_masked = glob.glob('/content/drive/MyDrive/academics/2ndyear/3/mini_project/dataset/AFDB_masked_face_dataset/*/*.jpg')
images_without_mask = glob.glob('/content/drive/MyDrive/academics/2ndyear/3/mini_project/dataset/AFDB_face_dataset/*/*.jpg')
total_files = len(images_masked)
print(total_files, len(images_without_mask))

In [ ]:
data = []
data_processed = []
labels = []

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
for _, imagePath in tqdm(enumerate(images_masked), total=len(images_masked)):
  image = load_img(imagePath, target_size=(64, 64))
  data.append(np.array(image))
  image = img_to_array(image)
  image = preprocess_input(image)
  data_processed.append(image)
  labels.append(1)

In [ ]:
len(data_processed)

In [ ]:
for _, imagePath in tqdm(enumerate(images_without_mask[:5000]), total=len(images_without_mask[:5000])):
  image = load_img(imagePath, target_size=(64, 64))
  data.append(np.array(image))
  image = img_to_array(image)
  image = preprocess_input(image)
  data_processed.append(image)
  labels.append(0)

In [ ]:
data = np.array(data, dtype="int")
data_processed = np.array(data_processed, dtype="float")
labels = np.array(labels)

In [ ]:
def hu_moments(image):
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the Hu Moments of the image as a feature
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [ ]:
def haralick(image):
    # Compute the haralick texture feature vector from the image
    haralick = mahotas.features.haralick(image).mean(axis=0)
    return haralick

In [ ]:
def histogram(image):
    # Compute the histogram of the image as a feature
    hist = cv2.calcHist([image], [0], None, [64], [0, 64])
    cv2.normalize(hist, hist)
    hist = hist.flatten()
    return hist

In [ ]:
def getFeatures(image):
    return np.hstack([histogram(image), haralick(image), hu_moments(image)])

In [ ]:
data_features = []

In [ ]:
data_processed[0]

In [ ]:
for img in data:
  data_features.append(getFeatures(img))


In [ ]:
data_features[0].shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def scale_data(features):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Normalize The feature vectors...
    mask_features = scaler.fit_transform(features)
    return mask_features

In [ ]:
data_features = scale_data(data_features)

In [ ]:
with open('/content/drive/MyDrive/academics/2ndyear/3/mini_project/img_csv/features.csv', 'w') as FOUT:
    np.savetxt(FOUT, data_features)

In [ ]:
with open('/content/drive/MyDrive/academics/2ndyear/3/mini_project/img_csv/labels.csv', 'w') as FOUT:
    np.savetxt(FOUT, labels)

In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/academics/2ndyear/3/mini_project/img_csv/features_test.pkl', 'wb') as f:
        pickle.dump(data_features, f)

In [ ]:
with open('/content/drive/MyDrive/academics/2ndyear/3/mini_project/img_csv/labels.pkl', 'wb') as f:
        pickle.dump(labels, f)

In [ ]:
with open('/content/drive/MyDrive/academics/2ndyear/3/mini_project/img_csv/data_preprocessed.pkl', 'wb') as f:
        pickle.dump(data_processed, f)